# Imports

In [1]:
import torch, torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
print(f"PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}")

PyTorch version: 1.13.1
torchvision version: 0.14.1


# Loading Data

In [3]:
to_tensor = transforms.ToTensor()

In [4]:
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=to_tensor)
test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=to_tensor)

In [5]:
# train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
# test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True)

In [6]:
BATCH_SIZE = 32
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(784, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        return x
    
model0 = Model0()
model0

Model0(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=784, out_features=10, bias=True)
)

In [8]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model0.parameters(), lr=0.001)

In [9]:
from torch_train import TorchTrain
from torchmetrics import Accuracy

In [10]:
acc = Accuracy(task = 'multiclass', num_classes = 10)

In [11]:
acc._get_name()


'MulticlassAccuracy'

In [12]:
type(acc)

torchmetrics.classification.accuracy.MulticlassAccuracy

In [13]:
metrics = {
    'accuracy': acc
}

In [14]:
tt = TorchTrain(model0, optimizer, loss, metrics)

In [15]:
acc._get_name()

'MulticlassAccuracy'

In [16]:
acc.__str__()

'MulticlassAccuracy()'

In [17]:
tt.fit(train_loader, test_loader, 5)

Epoch  1/ 5 Batch 1875/1875[====================] Train loss: 0.5902 | Train Accuracy: 0.8032
Epoch    1/   5[========------------] Test loss: 0.5031 | Test Accuracy: 0.8292
Epoch  2/ 5 Batch 1875/1875[====================] Train loss: 0.4607 | Train Accuracy: 0.8430
Epoch    2/   5[============--------] Test loss: 0.4768 | Test Accuracy: 0.8363
Epoch  3/ 5 Batch 1875/1875[====================] Train loss: 0.4325 | Train Accuracy: 0.8515
Epoch    3/   5[================----] Test loss: 0.4628 | Test Accuracy: 0.8343
Epoch  4/ 5 Batch 1875/1875[====================] Train loss: 0.4224 | Train Accuracy: 0.8531
Epoch    4/   5[====================] Test loss: 0.4550 | Test Accuracy: 0.8415
Epoch  5/ 5 Batch 1875/1875[====================] Train loss: 0.4121 | Train Accuracy: 0.8566
Epoch    5/   5[========================] Test loss: 0.4583 | Test Accuracy: 0.8410


In [18]:
def train_step(model, loss_fn, optimizer, x, y):
    model.train()
    yhat = model(x)
    l = loss_fn(yhat, y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    return l.item()

def test_step(model, loss_fn, x, y):
    model.eval()
    yhat = model(x)
    l = loss_fn(yhat, y)
    return l.item()

In [19]:
def accuracy(model, x, y):
    model.eval()
    yhat = model(x)
    yhat = torch.argmax(yhat, dim=1)
    return torch.sum(yhat == y).item() / len(y)

In [20]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss = 0
    test_loss = 0
    for i, (x, y) in enumerate(train_loader):
        train_loss += train_step(model0, loss, optimizer, x, y)
        print(f"Epoch: {epoch+1:>2d}/{EPOCHS:>2d} | Batch: {i+1:>3d}/{len(train_loader):>3d} | Loss: {train_loss/(i+1):.4f}", end='\r')
    train_acc = accuracy(model0, x, y)
    for x, y in test_loader:
        test_loss += test_step(model0, loss, x, y)
    test_acc = accuracy(model0, x, y)
    train_loss /= len(train_loader)
    test_loss /= len(test_loader)
    print(f"Epoch: {epoch+1:>2d}/{EPOCHS:>2d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

Epoch:  1/ 5 | Train Loss: 0.4069 | Train Acc: 1.0000 | Test Loss: 0.4584 | Test Acc: 0.8750
Epoch:  2/ 5 | Train Loss: 0.4010 | Train Acc: 0.8125 | Test Loss: 0.4449 | Test Acc: 0.8750
Epoch:  3/ 5 | Train Loss: 0.3982 | Train Acc: 0.7812 | Test Loss: 0.4456 | Test Acc: 0.8750
Epoch:  4/ 5 | Train Loss: 0.3937 | Train Acc: 0.8125 | Test Loss: 0.4428 | Test Acc: 0.8750
Epoch:  5/ 5 | Train Loss: 0.3908 | Train Acc: 0.9062 | Test Loss: 0.4607 | Test Acc: 0.8125


In [21]:
def predict(model, x):
    model.eval()
    yhat = model(x)
    yhat = torch.argmax(yhat, dim=1)
    return yhat

In [22]:
def plot_sample(x, y, yhat, classes):
    plt.imshow(x.squeeze(), cmap='binary')
    plt.title(f"Label: {classes[y]}\nPrediction: {classes[yhat]}")
    plt.axis('off')

x, y = test_data[0]
yhat = predict(model0, x.unsqueeze(0))
plot_sample(x, y, yhat, classes)

NameError: name 'classes' is not defined

In [ ]:
class Conv1(nn.Module):
    def __init__(self, color_channel: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=color_channel, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*7*7, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

In [ ]:
conv_model = Conv1(1, 10, 10)
conv_model

Conv1(
  (block_1): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=490, out_features=10, bias=True)
  )
)

In [ ]:
conv_loss = nn.CrossEntropyLoss()
conv_optimizer = torch.optim.Adam(conv_model.parameters(), lr=0.001)

EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss = 0
    test_loss = 0
    for i, (x, y) in enumerate(train_loader):
        train_loss += train_step(conv_model, conv_loss, conv_optimizer, x, y)
        print(f"Epoch: {epoch+1:>2d}/{EPOCHS:>2d} | Batch: {i+1:>3d}/{len(train_loader):>3d} | Loss: {train_loss/(i+1):.4f}", end='\r')
    train_acc = accuracy(conv_model, x, y)
    for x, y in test_loader:
        test_loss += test_step(conv_model, conv_loss, x, y)
    test_acc = accuracy(conv_model, x, y)
    train_loss /= len(train_loader)
    test_loss /= len(test_loader)
    print(f"Epoch: {epoch+1:>2d}/{EPOCHS:>2d} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")

Epoch:  1/ 5 | Train Loss: 0.5193 | Train Acc: 0.7812 | Test Loss: 0.4016 | Test Acc: 0.8750
Epoch:  2/ 5 | Train Loss: 0.3437 | Train Acc: 0.8438 | Test Loss: 0.3402 | Test Acc: 0.8750
Epoch:  3/ 5 | Train Loss: 0.3028 | Train Acc: 0.9688 | Test Loss: 0.3310 | Test Acc: 0.9375
Epoch:  4/ 5 | Train Loss: 0.2796 | Train Acc: 0.9062 | Test Loss: 0.2960 | Test Acc: 0.8125
Epoch:  5/ 5 | Train Loss: 0.2632 | Train Acc: 0.9375 | Test Loss: 0.2830 | Test Acc: 0.8125
